# Assignment 3: Decision Trees from scratch

__NOTE__ _This assignment is only required for students in 8515; students in 4510 may optionally complete it for extra credit._

For this assignment, your goal is to implement the ID3 algorithm (greedy information-gain based decision tree induction).  You should use the Congressional Voting Records data set, which I will show you how to load below.  This is a discrete multivariate dataset, which is well suited to decision trees.

Students in 4515 may optionally work in groups of 2 or 3 on this assignment (extra credit earned will be split equally); students in 8510 should work alone.

_Note that while scikit-learn does have decision trees, SKLearn in general doesn't really handle discrete data well, so the library version may not produce exactly the same trees that your algorithm will.  Also, since SKLearn expects numeric vector data, you'll need to do extra pre-processing before it will accept this data set._

Implement and test your algorithm; you should print the actual tree (a simple text-based visualization is fine), as well as calculating classification accuracy on held-out testing data.

As a reminder, please ensure that your notebook works correctly when you "restart kernel and run all."

In [2]:
# CSC 4510 - Machine Learning
# Assignment 3
# Scaffolding by Dr. Ben Mitchell
# Assignment completed by: <YOUR NAME(S) HERE>
# Resources used: 
#   <List any resources you used beyond the ones posted on Blackboard>
#   <This can include books, websites, other students, etc.>

## Read the data

The first thing we need to do is read the data from disk.  We'll use a library called Pandas to do that; it's really good for reading things like excel spreadsheets and comma-separated-value (CSV) data files.

In [1]:
# this time we'll use Pandas for reading our data file (it's great for CSVs)
import pandas as pd
# we'll also still want numpy
import numpy as np
from numpy import log2 as log

### The Dataset
#### For this assignment, you'll be using a dataset covering congressional voting behavior.  The original version is downloadable from the UCI machine learning repository (https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records), but you can just grab a copy from Blackboard.  Here's the dataset description:

1. Title: 1984 United States Congressional Voting Records Database

2. Source Information:
    (a) Source:  Congressional Quarterly Almanac, 98th Congress, 
                 2nd session 1984, Volume XL: Congressional Quarterly Inc. 
                 Washington, D.C., 1985.
    (b) Donor: Jeff Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
    (c) Date: 27 April 1987 

3. Past Usage
   - Publications
     1. Schlimmer, J. C. (1987).  Concept acquisition through 
        representational adjustment.  Doctoral dissertation, Department of 
        Information and Computer Science, University of California, Irvine, CA.
        -- Results: about 90%-95% accuracy appears to be STAGGER's asymptote
     - Predicted attribute: party affiliation (2 classes)

4. Relevant Information:
      This data set includes votes for each of the U.S. House of
      Representatives Congressmen on the 16 key votes identified by the
      CQA.  The CQA lists nine different types of votes: voted for, paired
      for, and announced for (these three simplified to yea), voted
      against, paired against, and announced against (these three
      simplified to nay), voted present, voted present to avoid conflict
      of interest, and did not vote or otherwise make a position known
      (these three simplified to an unknown disposition).

5. Number of Instances: 435 (267 democrats, 168 republicans)

6. Number of Attributes: 16 + class name = 17 (all Boolean valued)

7. Attribute Information:
   1. Class Name: 2 (democrat, republican)
   2. handicapped-infants: 2 (y,n)
   3. water-project-cost-sharing: 2 (y,n)
   4. adoption-of-the-budget-resolution: 2 (y,n)
   5. physician-fee-freeze: 2 (y,n)
   6. el-salvador-aid: 2 (y,n)
   7. religious-groups-in-schools: 2 (y,n)
   8. anti-satellite-test-ban: 2 (y,n)
   9. aid-to-nicaraguan-contras: 2 (y,n)
  10. mx-missile: 2 (y,n)
  11. immigration: 2 (y,n)
  12. synfuels-corporation-cutback: 2 (y,n)
  13. education-spending: 2 (y,n)
  14. superfund-right-to-sue: 2 (y,n)
  15. crime: 2 (y,n)
  16. duty-free-exports: 2 (y,n)
  17. export-administration-act-south-africa: 2 (y,n)

8. Missing Attribute Values: Denoted by "?"

   NOTE: It is important to recognize that "?" in this database does 
         not mean that the value of the attribute is unknown.  It 
         means simply, that the value is not "yea" or "nay" (see 
         "Relevant Information" section above).

In [2]:
# since the data file has no header, we need to define "names" for the columns
# since the class label (political party in this case) is first, we'll assign
# that name, and then just label the various votes as vN, where N is a number
colNames = ['party']
for i in range(16):
    colNames.append('v'+str(i))
    
# actually read the data, then take a look at it
raw = pd.read_csv('house-votes-84.data', header=None, names=colNames )
raw

,party,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
6,democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
7,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
9,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?


In [3]:
# as usual, length will give us the number of examples
len(raw)

435

## Split the data into training and testing sets

Next, we need to split the data into train and test sets, and then break off the labels from the feature vectors.  We'll also convert from Pandas style tables to numpy style arrays.

In [4]:
# let's shuffle this and split it into train and test sets:
shuffled = raw.sample(frac=1) # randomly re-order the examples
trainFrac = 0.7 # 70%/30% train/test split
trainCount = int(len(raw) * trainFrac)
train = shuffled[:trainCount]
test = shuffled[trainCount:]

In [5]:
print("train example count:", len(train), ", test example count:", len(test))

train example count: 304 , test example count: 131


In [6]:
test

,party,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15
274,republican,y,n,n,y,y,n,y,n,n,y,n,n,n,y,y,y
200,democrat,n,n,y,n,n,n,y,y,y,n,n,n,n,y,y,y
284,democrat,n,n,y,n,n,y,y,y,y,y,y,n,n,n,?,y
30,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,n
299,democrat,n,y,y,n,n,y,y,y,y,y,n,n,y,y,y,y
215,democrat,n,y,y,y,y,y,n,n,n,y,y,y,y,y,y,?
291,democrat,y,n,y,n,n,y,y,y,y,y,n,?,n,y,n,y
328,democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,n,y
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
282,republican,y,n,n,y,y,y,n,n,n,y,n,?,y,y,n,n


In [7]:
test['party'].values

array(['republican', 'democrat', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'democrat', 'republican', 'republican',
       'democrat', 'republican', 'democrat', 'republican', 'republican',
       'republican', 'democrat', 'republican', 'republican', 'republican',
       'republican', 'republican', 'democrat', 'democrat', 'democrat',
       'republican', 'democrat', 'democrat', 'republican', 'republican',
       'democrat', 'republican', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'democrat', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'republican', 'democrat', 'republican',
       'republican', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'republican', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'democrat', 'republican', 'democrat',


In [8]:
# split off class labels and convert to numpy array
trainLabels = train['party'].values
testLabels = test['party'].values
len(trainLabels)

304

In [9]:
# let's take a look at what we got
testLabels

array(['republican', 'democrat', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'democrat', 'republican', 'republican',
       'democrat', 'republican', 'democrat', 'republican', 'republican',
       'republican', 'democrat', 'republican', 'republican', 'republican',
       'republican', 'republican', 'democrat', 'democrat', 'democrat',
       'republican', 'democrat', 'democrat', 'republican', 'republican',
       'democrat', 'republican', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'democrat', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'republican', 'democrat', 'republican',
       'republican', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'republican', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'democrat', 'republican', 'democrat',


In [10]:
# find all the distinct options:
classLabels = np.unique(trainLabels)
classLabels

array(['democrat', 'republican'], dtype=object)

In [11]:
# now remove the labels from the feature vectors and convert to numpy array
trainFeatures = train.drop(['party'], axis=1).values
testFeatures = test.drop(['party'], axis=1).values
trainFeatures

array([['y', 'y', 'y', ..., 'n', 'y', '?'],
       ['y', 'n', 'n', ..., 'y', 'n', 'y'],
       ['n', '?', 'y', ..., 'n', 'y', 'y'],
       ...,
       ['y', '?', 'y', ..., 'n', 'y', 'y'],
       ['n', 'y', 'y', ..., 'y', 'n', 'y'],
       ['n', 'y', 'y', ..., 'y', 'n', '?']], dtype=object)

In [12]:
# we can look at a row by index
trainFeatures[0]

array(['y', 'y', 'y', 'n', 'n', 'n', 'y', 'y', 'y', 'n', 'y', 'n', 'n',
       'n', 'y', '?'], dtype=object)

In [13]:
# we can look at a single feature with a double index (row, col)
trainFeatures[0][0]

'y'

In [14]:
# total number of training examples
numExamples = len(trainFeatures)
numExamples

304

In [15]:
# number of features in the first example (should be the same for all examples)
numFeatures = len(trainFeatures[0])
numFeatures

16

In [16]:
# now let's find the unique feature values
featureValues = np.unique(trainFeatures)
featureValues

array(['?', 'n', 'y'], dtype=object)

# Implement Decision Tree #

From here, you're on your own to implement an algorithm to train and test a decision tree on this dataset.

## Complete Concise Work This Time

In [33]:
uniqueLabels = np.unique(trainLabels)
len(uniqueLabels)

2

In [17]:
# checks if we have more then one unique label
def checkUnique(train):
  trainLabels = train[:, 0]
  uniqueLabels = np.unique(trainLabels)
  if len(uniqueLabels) > 1:
    return True
  else:
    return False

In [18]:
# This gives us all of the unique possible features that we can possibly split on
def getPossibleSplits(train):
  trainFeatures = train[:, 1:]
  possibleSplits = {}
  _, totalColumns = trainFeatures.shape
  for i in range(totalColumns):
    values = trainFeatures[:, i]
    uniqueValues = np.unique(values)
    possibleSplits[i] = uniqueValues
    
  return possibleSplits

In [19]:
# We are working with categorical data, so we are looking for matches with our split values
def splitData(train, splitColumn, splitValue):
  splitColValues = train[:, splitColumn]
  match = train[splitColValues == splitValue]
  noMatch = train[splitColValues != splitValue]
  return match, noMatch

In [20]:
def calcEntropy(train):
  trainLabels = train[:, 0]
  _, count = np.unique(trainLabels, return_counts=True)
  probability = count / count.sum()
  entropy = -sum(probability * np.log2(probability)) 
  return entropy

In [21]:
def calcInfoGain(match, noMatch):
  n = len(match) + len(noMatch)
  pMatch = len(match) / n
  pNoMatch = len(noMatch) / n
  gain =  (pMatch * calcEntropy(match) + pNoMatch * calcEntropy(noMatch))
  return gain

In [22]:
def getBestSplit(train, possibleSplits):  
  infoGain = 10000000 # just needs to be outrageously sized for the declaration
  for colIndex in possibleSplits:
      for value in possibleSplits[colIndex]:
          match, noMatch = splitData(train, colIndex, value)
          currentInfoGain = calcInfoGain(match, noMatch)

          if currentInfoGain <= infoGain:
              infoGain = currentInfoGain
              bestSplitCol = colIndex
              bestSplitVal = value

  return bestSplitCol, bestSplitVal

In [23]:
def classify(train):   
  trainLabels = train[:, 0]
  uniqueLabels, count = np.unique(trainLabels, return_counts=True)
  index = count.argmax()
  classification = uniqueLabels[index]
  return classification

In [34]:
def buildTree(train, count):
  if count == 0: #initialize data and retrieve information
    global columnHeaders # save the column names
    columnHeaders = train.columns
    data = train.values
    count += 1
  else:
    data = train
  
  if checkUnique(data) != True: # base case for unique labels
    classification = classify(data)
    return classification
  else:
    possibleSplits = getPossibleSplits(data) # get the possible splits
    splitCol, splitVal = getBestSplit(data, possibleSplits) # check to see what one is the best
    match, noMatch = splitData(data, splitCol, splitVal) # split the data
    
    if len(match) == 0 or len(noMatch) == 0: # we're done
      classification = classify(data)
      return classification
    
    # building of the tree, tells what we split on, what the column was that we split on
    feature = columnHeaders[splitCol]
    q = "{} = {}".format(feature, splitVal) # makes the tree more readable by defining what we split on
    subTree = {q: []}
    yes = buildTree(match, count)
    no = buildTree(noMatch, count)

    if yes == no: # get the same answer whether it is a yes or no, we'll just say it is yes
        subTree = yes
    else:
        subTree[q].append(yes)
        subTree[q].append(no)

    return subTree

In [36]:
tree = buildTree(train, 0)
tree

{'v3 = y': [{'v9 = n': [{'v10 = y': [{'v12 = y': [{'v14 = y': ['democrat',
          {'v2 = y': ['democrat',
            {'v11 = y': [{'v1 = n': ['democrat', 'republican']},
              'democrat']}]}]},
        'democrat']},
      {'v14 = y': [{'v13 = y': [{'v11 = y': ['republican', 'democrat']},
          'republican']},
        {'v2 = y': [{'v6 = y': ['republican', 'democrat']}, 'republican']}]}]},
    'republican']},
  {'v2 = ?': [{'v11 = n': ['democrat',
      {'v13 = ?': [{'v8 = ?': ['republican', 'democrat']}, 'republican']}]},
    {'v2 = n': [{'v10 = y': ['democrat',
        {'v12 = y': ['democrat',
          {'v14 = n': ['republican',
            {'v11 = n': ['democrat', 'republican']}]}]}]},
      'democrat']}]}]}

In [39]:
testResults = test.iloc[0]

In [78]:
# q and a are just question and answer. Since the tree is built on the question answer idea.
def traverseTree(testResults, tree):
  q = list(tree.keys())[0] 
  feature, equal, value = q.split(" ") # equal is a place to put the equal sign
  
  if str(testResults[feature]) == value:
      a = tree[q][0]
  else:
      a = tree[q][1]

  # base case
  if not isinstance(a, dict):
      return a

  # recursive part
  else:
      residualTree = a
      return traverseTree(testResults, residualTree)

In [43]:
traverseTree(testResults, tree)

'republican'

In [75]:
def calcAccuracy(data, tree):
  data["classifiedValue"] = data.apply(traverseTree, args=(tree,), axis=1)
  data["correct"].append(data["classifiedValue"] == data["party"])
  accuracy = data["correct"].mean()
  return accuracy

In [76]:
acc = calcAccuracy(test, tree)
acc

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.9389312977099237